In [ ]:
#| default_exp testing

In [ ]:
%pip install -q langchain_core
%pip install -q langchain_openai
%pip install -q psycopg2-binary

In [ ]:

from langchain_core.prompts import PromptTemplate

In [ ]:
firstPrompt = PromptTemplate.from_template("""Return one ore more json objects together in an array to describe the jobs in a job posting. If there is only one job posting presented here, return a json object containing a summary of every separate piece of information in the job posting. If the job describes multiple roles, create separate json objects for each role. Here is the job posting:\n{job_posting}""")

subsequentPrompt = PromptTemplate.from_template("""You will be returning one ore more json objects together in an array to describe the jobs in a job posting. Here are examples of the format of the json object you should return:\n{json_examples}\nHere are the key values currently being used to categorize jobs:\n{key_values}.\n If there is only one job posting presented here, return a json object containing a summary of every separate piece of information in the job posting. If the job describes multiple roles, create a separate json object for each role. Use the existing key values where they are relevant. If there is no relevant existing key, add new, relevant key to your json object:\n{job_posting}""")


In [ ]:
prompt  = subsequentPrompt.format(job_posting="This is a job posting", json_examples="[{\"role\": \"Software Engineer\", \"location\": \"San Francisco\", \"salary\": \"$120,000\"}, {\"role\": \"Data Scientist\", \"location\": \"New York\", \"salary\": \"$150,000\"}]", key_values="role, location, salary")

print(prompt)

You will be returning one ore more json objects together in an array to describe the jobs in a job posting. Here are examples of the format of the json object you should return:
 [{"role": "Software Engineer", "location": "San Francisco", "salary": "$120,000"}, {"role": "Data Scientist", "location": "New York", "salary": "$150,000"}]

Here are the key values currently being used to categorize jobs:

role, location, salary.

 If there is only one job posting presented here, return a json object containing a summary of every separate piece of information in the job posting. If the job describes multiple roles, create a separate json object for each role. Use the existing key values where they are relevant. If there is no relevant existing key, add new, relevant key to your json object: This is a job posting


In [ ]:
import json

# Opening JSON file
f = open('../posts.json')

# returns JSON object as 
# a dictionary
posts = json.load(f)

In [ ]:
# | export
import psycopg2

def connect():
    conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
    cur = conn.cursor()
    return conn, cur

conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
cur = conn.cursor()

cur.execute(
    """CREATE TABLE IF NOT EXISTS parsedPosts (
        id SERIAL PRIMARY KEY, 
        hnuser TEXT, 
        date TIMESTAMP, 
        comment TEXT,
        embedding vector(1536)
        );"""
    )

conn.commit()    

In [ ]:
from langchain_openai import ChatOpenAI

post = posts[0]

msg = firstPrompt.format(job_posting = post['comment'])

model = ChatOpenAI()

response = model.invoke(msg)

In [ ]:
# Parse the JSON string to a Python dictionary
initial_parsed_datum = json.loads(response.content)
parsed_data = [ initial_parsed_datum ]

In [ ]:
import  json

result = {}
keys = []

examplesStr = json.dumps(initial_parsed_datum)
postItems = posts[:2]

for i in range(len(postItems)):
    if i != 0:
        msg = subsequentPrompt.format(job_posting = postItems[i]['comment'], json_examples = examplesStr, key_values = ", ".join(keys))

        print(msg)
        resp = model.invoke(msg)
        parsed_datum = json.loads(resp.content)
        for  job in  parsed_datum:
            parsed_data.append(job)

        # We don't care much for the shape of the json object the machine generates, as long as it remains uniform. So, we use the first 5 outputs as examples for the rest.
        if i < 5:
            itemStr = map(lambda x: json.dumps(x), parsed_data)
            examplesStr += " ".join(itemStr)

        if i == 4 or i == 0:
            print('\n\n', msg)
            print ('\n\n', examplesStr)

        print(parsed_data)




You will be returning one ore more json objects together in an array to describe the jobs in a job posting. Here are examples of the format of the json object you should return:
[{"job_title": "Senior Software Engineer", "company": "Irth Solutions", "description": "Build software that improves resilience and reduces risk to critical infrastructure. Handle millions of requests a day and coordinate activities between infrastructure providers, locators, excavators, and other stakeholders throughout North America.", "technical_requirements": ["C# development experience using .NET Core, WebAPI, and Entity Framework", "Experience building browser applications using Angular and Typescript", "Strong relational database experience with Postgres (preferred) or MS SQL Server including SQL, database functions, indexing, and performance tuning", "Kubernetes, Docker, Linux"], "apply_link": "https://apply.workable.com/irth-solutions/j/33AD66BA3A/"}]
Here are the key values currently being used to cat

In [ ]:
postItems

[{'hnuser': 'grantmuller',
  'date': '2024-07-08T22:39:26',
  'comment': 'At Irth Solutions we build software that improve resilience and reduce risk to critical infrastructure that millions of people and businesses rely on each day. Our systems handle millions of requests a day, coordinating activities between critical infrastructure providers, contract locators, excavators, and a variety of other stakeholders throughout North America.\nWe’re growing our team and are in immediate need for a Senior Software Engineer. Here are some notes on the technical requirements:\n* C# development experience using .NET Core, WebAPI, and Entity Framework\n* Experience building browser applications using Angular and Typescript\n* Strong relational database experience with Postgres (preferred) or MS SQL Server including SQL, database functions, indexing, and performance tuning\n* Kubernetes, Docker, Linux\nApply here: https://apply.workable.com/irth-solutions/j/33AD66BA3A/\nWe’re growing, so new posit

We split the iteration over the request so we don't do it unnecessarily

In [ ]:

for i in range(len(parsed_data)):
        # keys needs to be a unique list of keys
        # so we can't just add the keys of the new datum
        # we need to add only the keys that are not already in keys

        jobdesc = parsed_data[i]
        for job in jobdesc:

            print(job)
            # jobObj = json.loads(job)
            # parsed_data_keys = list(jobObj.keys()) 
            # for key in parsed_data_keys:
            #     if key not in keys:
            #         keys.append(key)
            #         # curs.execute("ALTER TABLE parsedPosts ADD COLUMN IF NOT EXISTS %s TEXT", (key,))
            #         conn.commit()

            # keyStr = ", ".join(keys)

            # curs.execute("INSERT INTO posts (hnuser, date, comment, %s) VALUES (%s, %s, %s, %s)", (posts[i]['hnuser'], posts[i]['date'], posts[i]['comment'], parsed_datum))

    

{'job_title': 'Senior Software Engineer', 'company': 'Irth Solutions', 'description': 'Build software that improves resilience and reduces risk to critical infrastructure. Handle millions of requests a day and coordinate activities between infrastructure providers, locators, excavators, and other stakeholders throughout North America.', 'technical_requirements': ['C# development experience using .NET Core, WebAPI, and Entity Framework', 'Experience building browser applications using Angular and Typescript', 'Strong relational database experience with Postgres (preferred) or MS SQL Server including SQL, database functions, indexing, and performance tuning', 'Kubernetes, Docker, Linux'], 'apply_link': 'https://apply.workable.com/irth-solutions/j/33AD66BA3A/'}
company
position
location
remote
employment_type
description
tech_stack
cloud_provider
contact_email
additional_information
company
position
location
remote
employment_type
description
tech_stack
cloud_provider
contact
more_info
co

In [ ]:
print(parsed_data)

[[[{'company': 'Irth Solutions', 'industry': 'Software development for critical infrastructure', 'software': 'C# development using .NET Core, WebAPI, and Entity Framework; browser applications using Angular and Typescript', 'database': 'Strong relational database experience with Postgres (preferred) or MS SQL Server including SQL, database functions, indexing, and performance tuning', 'tools': 'Kubernetes, Docker, Linux', 'position': 'Senior Software Engineer', 'application_link': 'https://apply.workable.com/irth-solutions/j/33AD66BA3A/', 'growth_opportunities': 'New positions will open up frequently'}]], [{'company': 'Tandem Health', 'position': 'Software Engineers', 'location': 'On-site in Stockholm, Sweden', 'employment_type': 'Full time', 'description': "At Tandem Health we're building a clinician copilot to allow clinicians to focus on care rather than administration. We’re building, launching and iterating quickly, already at $300k ARR (+35% MoM) with over 50 care providers using

In [ ]:

# let's see how frequently each key appears
tally = {}
keys
for key in keys:
#     result[key] = [datum[key] for datum in parsed_data]
    tally[key] = sum([1 for datum in parsed_data if key in datum])

# result, tally

In [ ]:
parsed_data

[[[{'company': 'Irth Solutions',
    'industry': 'Software development for critical infrastructure',
    'software': 'C# development using .NET Core, WebAPI, and Entity Framework; browser applications using Angular and Typescript',
    'database': 'Strong relational database experience with Postgres (preferred) or MS SQL Server including SQL, database functions, indexing, and performance tuning',
    'tools': 'Kubernetes, Docker, Linux',
    'position': 'Senior Software Engineer',
    'application_link': 'https://apply.workable.com/irth-solutions/j/33AD66BA3A/',
    'growth_opportunities': 'New positions will open up frequently'}]],
 [{'company': 'Tandem Health',
   'position': 'Software Engineers',
   'location': 'On-site in Stockholm, Sweden',
   'employment_type': 'Full time',
   'description': "At Tandem Health we're building a clinician copilot to allow clinicians to focus on care rather than administration. We’re building, launching and iterating quickly, already at $300k ARR (+3

We add all the gathered keys to the table.

In [ ]:
for key in keys:
    cur.execute(f"ALTER TABLE parsedPosts ADD COLUMN IF NOT EXISTS  \"{key}\"  TEXT")
    conn.commit()

We create a SQL query that posts by our new keys

In [ ]:
sql_query = "INSERT INTO parsedPosts ("+ ", ".join(keys) +") VALUES (" + ", ".join(["%s" for key in keys]) + ")"

conn, cur = connect()

# for i in range(len(parsed_data)):
#     datum = parsed_data[i]
#     datum_keys = list(datum.keys())

#     for key in keys:
#         if key not in datum_keys:
#             datum[key] = None

#     values = [datum[key] for key in keys]
#     valuees = [str(value) for value in values]
#     print(values)
#     # cur.execute(sql_query, valuees)
#     # conn.commit()

cur.execute("SELECT * FROM parsedPosts")

cur.fetchall()

# cur.close()

[(1,
  None,
  None,
  None,
  None,
  None,
  'Irth Solutions',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'https://apply.workable.com/irth-solutions/j/33AD66BA3A/',
  'Software development for critical infrastructure',
  None,
  None,
  None,
  None,
  None,
  'Senior Software Engineer',
  None,
  None,
  None,
  None,
  None,
  None),
 (2,
  None,
  None,
  None,
  None,
  None,
  'Irth Solutions',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'https://apply.workable.com/irth-solutions/j/33AD66BA3A/',
  'Software development for critical infrastructure',
  None,
  None,
  None,
  None,
  None,
  'Senior Software Engineer',
  None,
  None,
  None,
  None,
  None,
  None),
 (3,
  None,
  None,
  None,
  None,
  None,
  'Irth Solutions',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'https://apply.workable.com/irth-solutions/j/33